In [ ]:
import sympy
from sympy import lambdify
import dill
dill.settings["recurse"] = True
from sympy import Rational as R
from tqdm.notebook import tqdm
import numpy as np

In [ ]:
(
    reference_x,
    reference_y,
    physical_x_0,
    physical_y_0,
    physical_x_1,
    physical_y_1,
    physical_x_2,
    reference_y_3,
) = sympy.symbols("x y x_0 y_0 x_1 y_1 x_2 y_2")

(
    physical_v11_1, physical_v11_2,
    physical_v12_1, physical_v12_2, 
    physical_v21_1, physical_v21_2, 
    physical_v22_1, physical_v22_2, 
    physical_v31_1, physical_v31_2, 
    physical_v32_1, physical_v32_2, 
) = sympy.symbols("v_(1:4)(1:3)_(1:3)")
f_interpolation = sympy.Matrix(sympy.symbols("f_0:3"))

## Define basis function

In [ ]:
monomial_basis = sympy.Matrix([
    1,
    reference_x,
    reference_y,
])

V = sympy.zeros(3, 3)

for i, basis in enumerate(monomial_basis):
    
    V[i, 0] = basis.subs({reference_x: 0, reference_y: 0})
    V[i, 1] = basis.subs({reference_x: 1, reference_y: 0})
    V[i, 2] = basis.subs({reference_x: 0, reference_y: 1})

lagrange_linear_basis = V.inv() @ monomial_basis

## Define mapping function

In [ ]:
monomial_basis = sympy.Matrix([
    1,
    reference_x,
    reference_y,
    reference_x**2,
    reference_x*reference_y,
    reference_y**2,
    reference_x**3,
    reference_x**2*reference_y,
    reference_x*reference_y**2,
    reference_y**3,
])

V = sympy.zeros(10, 10)

for i, basis in enumerate(monomial_basis):
    
    V[i, 0] = basis.subs({reference_x: 0, reference_y: 0})
    V[i, 1] = basis.subs({reference_x: 1, reference_y: 0})
    V[i, 2] = basis.subs({reference_x: 0, reference_y: 1})

    V[i, 3] = basis.diff(reference_x).subs({reference_x: 0, reference_y: 0})
    V[i, 4] = basis.diff(reference_y).subs({reference_x: 0, reference_y: 0})
    
    V[i, 5] = -basis.diff(reference_x).subs({reference_x: 1, reference_y: 0})
    V[i, 6] = (-basis.diff(reference_x).subs({reference_x: 1, reference_y: 0}) + +basis.diff(reference_y).subs({reference_x: 1, reference_y: 0}))
    
    V[i, 7] = -basis.diff(reference_y).subs({reference_x: 0, reference_y: 1})
    V[i, 8] = (+basis.diff(reference_x).subs({reference_x: 0, reference_y: 1}) + -basis.diff(reference_y).subs({reference_x: 0, reference_y: 1}))

    # center of mass
    V[i, 9] = basis.subs({reference_x: R(1,3), reference_y: R(1,3)})
    
mapping_basis = V.inv() @ monomial_basis

a1 = ((physical_x_1 - physical_x_0)**2 + (physical_y_1 - physical_y_0)**2)**0.5
a2 = ((physical_x_2 - physical_x_0)**2 + (reference_y_3 - physical_y_0)**2)**0.5
a3 = ((physical_x_0 - physical_x_1)**2 + (physical_y_0 - physical_y_1)**2)**0.5
a4 = ((physical_x_0 - physical_x_2)**2 + (physical_y_0 - reference_y_3)**2)**0.5

physical_v11_1 = (physical_x_1 - physical_x_0)# / a1
physical_v12_1 = (physical_x_2 - physical_x_0)# / a2
physical_v21_1 = (physical_x_0 - physical_x_1)# / a3
physical_v31_1 = (physical_x_0 - physical_x_2)# / a4

physical_v11_2 = (physical_y_1 - physical_y_0)# / a1
physical_v12_2 = (reference_y_3 - physical_y_0)# / a2
physical_v21_2 = (physical_y_0 - physical_y_1)# / a3
physical_v31_2 = (physical_y_0 - reference_y_3)# / a4


global_x = physical_x_0*mapping_basis[0] + physical_x_1*mapping_basis[1] + physical_x_2*mapping_basis[2] + \
    physical_v11_1*mapping_basis[3] + physical_v12_1*mapping_basis[4] + \
    physical_v21_1*mapping_basis[5] + physical_v22_1*mapping_basis[6] + \
    physical_v31_1*mapping_basis[7] + physical_v32_1*mapping_basis[8] + \
    R(1,3)*(physical_x_0 + physical_x_1 + physical_x_2)*mapping_basis[9]
    
    
global_y = physical_y_0*mapping_basis[0] + physical_y_1*mapping_basis[1] + reference_y_3*mapping_basis[2] + \
    physical_v11_2*mapping_basis[3] + physical_v12_2*mapping_basis[4] + \
    physical_v21_2*mapping_basis[5] + physical_v22_2*mapping_basis[6] + \
    physical_v31_2*mapping_basis[7] + physical_v32_2*mapping_basis[8] + \
    R(1,3)*(physical_y_0 + physical_y_1 + reference_y_3)*mapping_basis[9]
    

mapping_function = sympy.Matrix([global_x, global_y])


In [ ]:
t = np.linspace(0, 1, 60)
tx, ty = np.meshgrid(t, t, indexing="ij")

mask = ty <= 1 - tx
tx = tx[mask]
ty = ty[mask]

In [ ]:
import matplotlib.pyplot as plt
import plotly

In [ ]:
points = np.array([
    [0, 1],
    [3, 1],
    [0, 3]
]).astype(float)

v_22_1 = 0
v_22_2 = 6

v_32_1 = 6
v_32_2 = 0

iv = np.array([1, 1, 3]).astype(float)

In [ ]:
gx, gy = sympy.lambdify([
                reference_x,
                reference_y,
                physical_x_0,
                physical_y_0,
                physical_x_1,
                physical_y_1,
                physical_x_2,
                reference_y_3,
                physical_v22_1, physical_v22_2, 
                physical_v32_1, physical_v32_2, 
                
                ], mapping_function)(
                                        tx, ty, 
                                        points[0, 0], points[0, 1],
                                        points[1, 0], points[1, 1],
                                        points[2, 0], points[2, 1],
                                        v_22_1, v_22_2,
                                        v_32_1, v_32_2,                                        
                                     ).squeeze()

In [ ]:
plt.scatter(gx, gy)
plt.axis("equal")
plt.show()

In [ ]:
gz = iv@sympy.lambdify([reference_x, reference_y], lagrange_linear_basis)(tx, ty).squeeze()

In [ ]:
plot_data = [
    plotly.graph_objects.Mesh3d(
                                x=gx, 
                                y=gy,
                                z=gz,
                                intensity=gz
                                )        
    ]
fig = plotly.graph_objects.Figure(data=plot_data)
fig.update_layout()

In [ ]:
J_cofactor_T = mapping_function.jacobian([reference_x, reference_y]).cofactor_matrix().T

In [ ]:
J = J_cofactor_T / J_cofactor_T.det()

In [ ]:
u = sympy.Matrix([sympy.Function("u")(reference_x, reference_y)])
v = sympy.Matrix([sympy.Function("v")(reference_x, reference_y)])

In [ ]:
grad_u = u.jacobian([reference_x, reference_y])@J_cofactor_T
grad_v = v.jacobian([reference_x, reference_y])@J_cofactor_T

weak_form = grad_u@grad_v.T

In [ ]:
N = len(lagrange_linear_basis)

weak_form_functional = [[0 for i in range(N)] for j in range(N)]
weak_form_right_part = [[0 for i in range(N)] for j in range(N)]

for idx, jdx in tqdm([(idx, jdx) for idx in range(N) for jdx in range(N)]):
    first = lagrange_linear_basis[idx]
    second = lagrange_linear_basis[jdx]

    A = weak_form.subs({u[0]:first, v[0]:second}).doit()
    B = first * second
    
    weak_form_functional[idx][jdx] = A
    weak_form_right_part[idx][jdx] = B

weak_form_functional = sympy.Matrix(weak_form_functional) / J_cofactor_T.det()**2 * abs(J_cofactor_T.det())
weak_form_right_part = sympy.Matrix(weak_form_right_part) * abs(J_cofactor_T.det())

In [ ]:
lambdify_symbols = [
    reference_x, reference_y,
    physical_x_0,
    physical_y_0,
    physical_x_1,
    physical_y_1,
    physical_x_2,
    reference_y_3,
    physical_v22_1,
    physical_v22_2,
    physical_v32_1,
    physical_v32_2,
]

In [ ]:
weak_form_right_part = weak_form_right_part @ f_interpolation

In [ ]:
weak_form_functional_lambdified = sympy.lambdify(lambdify_symbols, weak_form_functional, cse=True)

In [ ]:
weak_form_right_part_lambdified = sympy.lambdify([*lambdify_symbols, *f_interpolation], weak_form_right_part, cse=True)

In [ ]:
quadrature_weights = np.array([
0.090817990382754,
0.036725957756467,
0.036725957756467,
0.036725957756467,
0.072757916845420,
0.072757916845420,
0.072757916845420,
0.072757916845420,
0.072757916845420,
0.072757916845420,
0.028327242531057,
0.028327242531057,
0.028327242531057,
0.028327242531057,
0.028327242531057,
0.028327242531057,
0.009421666963733,
0.009421666963733,
0.009421666963733,
0.009421666963733,
0.009421666963733,
0.009421666963733,
0.045321059435528,
0.045321059435528,
0.045321059435528,
]) / 2


quadrature_points = np.array([
[0.333333333333333,0.333333333333333],
[0.485577633383657,0.028844733232685],
[0.485577633383657,0.485577633383657],
[0.028844733232685,0.485577633383657],
[0.141707219414880,0.550352941820999],
[0.307939838764121,0.141707219414880],
[0.550352941820999,0.307939838764121],
[0.307939838764121,0.550352941820999],
[0.141707219414880,0.307939838764121],
[0.550352941820999,0.141707219414880],
[0.025003534762686,0.728323904597411],
[0.246672560639903,0.025003534762686],
[0.728323904597411,0.246672560639903],
[0.246672560639903,0.728323904597411],
[0.025003534762686,0.246672560639903],
[0.728323904597411,0.025003534762686],
[0.009540815400299,0.923655933587500],
[0.066803251012200,0.009540815400299],
[0.923655933587500,0.066803251012200],
[0.066803251012200,0.923655933587500],
[0.009540815400299,0.066803251012200],
[0.923655933587500,0.009540815400299],
[0.109481575485037,0.781036849029926],
[0.109481575485037,0.109481575485037],
[0.781036849029926,0.109481575485037],
])

In [ ]:
def weak_form_functional_lambdified_quadrature(x_1, y_1, x_2, y_2, x_3, y_3, v_22_1, v_22_2, v_32_1, v_32_2):
    
    return weak_form_functional_lambdified(*quadrature_points.T, x_1, y_1, x_2, y_2, x_3, y_3, v_22_1, v_22_2, v_32_1, v_32_2)@quadrature_weights

In [ ]:
def weak_form_right_part_lambdified_quadrature(x_1, y_1, x_2, y_2, x_3, y_3, v_22_1, v_22_2, v_32_1, v_32_2,
                                               f_1, f_2, f_3):
    
    return weak_form_right_part_lambdified(*quadrature_points.T, x_1, y_1, x_2, y_2, x_3, y_3, v_22_1, v_22_2, v_32_1, v_32_2,
                                           f_1, f_2, f_3)@quadrature_weights

In [ ]:
interpolation_function = sympy.lambdify([*lambdify_symbols, *f_interpolation], (lagrange_linear_basis@f_interpolation.T)[0, 0], cse=True)

In [ ]:
mapping_function_lambdified = sympy.lambdify(lambdify_symbols, mapping_function[:, 0], cse=True)

In [ ]:
dill.dump(weak_form_functional_lambdified_quadrature, open("../calculations/lagrange_linear_weak_form_functional_cubic_hermite", "wb"))
dill.dump(weak_form_right_part_lambdified_quadrature, open("../calculations/lagrange_linear_weak_form_right_part_cubic_hermite", "wb"))
dill.dump(interpolation_function, open("../calculations/lagrange_linear_basis", "wb"))
dill.dump(mapping_function_lambdified, open("../calculations/lagrange_linear_mapping_function", "wb"))